You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-08-12 17:56:54,390 - 140610451331968 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Unveiling the Latest Trends in Artificial Intelligence 

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in various industries
- Brief overview of the latest trends in AI

II. Key Trends in Artificial Intelligence
A. Machine Learning and Deep Learning advancements
B. Natural Language Processing developments
C. AI-powered automation in business processes
D. Ethical considerations in AI development
E. The rise of AI in healthcare and personalized medici

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends in Artificial Intelligence

Artificial Intelligence (AI) continues to be a driving force in transforming industries worldwide, showcasing its ability to replicate human intelligence and execute tasks that traditionally necessitate human intervention. From the healthcare sector to finance, AI is reshaping business operations and customer interactions. The latest trends in AI highlight technological advancements that are pushing the boundaries of what can be achieved. This blog post will explore the key trends in AI, shed light on the major players in the industry, present notable news, and evaluate the impact of AI on society and the workforce.

## Key Trends in Artificial Intelligence

### Advancements in Machine Learning and Deep Learning
Machine Learning and Deep Learning have emerged as pivotal components of AI progress, with algorithms becoming increasingly intricate and adept at handling complex operations.

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends in Artificial Intelligence

Artificial Intelligence (AI) continues to be a driving force in transforming industries worldwide, showcasing its ability to replicate human intelligence and execute tasks that traditionally necessitate human intervention. From the healthcare sector to finance, AI is reshaping business operations and customer interactions. The latest trends in AI highlight technological advancements that are pushing the boundaries of what can be achieved. This blog post will explore the key trends in AI, shed light on the major players in the industry, present notable news, and evaluate the impact of AI on society and the workforce.

## Key Trends in Artificial Intelligence

### Advancements in Machine Learning and Deep Learning
Machine Learning and Deep Learning have emerged as pivotal components of AI progress, with algorithms becoming increasingly intricate and adept at handling complex operations. Organizations are leveraging these technologies to enhance decision-making processes and elevate user experiences.

### Developments in Natural Language Processing
Significant strides have been made in Natural Language Processing (NLP), enabling machines to comprehend and generate human language. This progress has paved the way for virtual assistants and chatbots that engage with users in a more natural and intuitive manner.

### AI-Powered Automation in Business Operations
AI-driven automation is revolutionizing business processes by automating repetitive tasks, thereby freeing up human resources to concentrate on strategic endeavors. This shift towards automation has resulted in enhanced efficiency and productivity across various industries.

### Ethical Considerations in AI Progression
As AI continues to evolve, ethical considerations have become integral to its development. Issues such as algorithmic bias, data privacy, and transparency in decision-making are being addressed to ensure the responsible development and deployment of AI technologies.

### The Emergence of AI in Healthcare and Personalized Medicine
AI is making significant inroads in the healthcare sector, from disease diagnosis to personalized treatment plans. The integration of AI in healthcare is revolutionizing patient care and enhancing health outcomes.

## Key Players in the AI Industry

### Google AI
Google AI stands at the forefront of AI research and development, with initiatives spanning healthcare, natural language comprehension, and autonomous systems.

### IBM Watson
IBM Watson leads the field in cognitive computing, offering solutions in data analytics, machine learning, and natural language processing.

### Microsoft Azure AI
Microsoft Azure AI provides a comprehensive suite of AI services and tools for businesses to construct, deploy, and manage AI solutions.

### Amazon Web Services (AWS) AI
AWS AI offers a diverse array of AI services, including machine learning, natural language processing, and computer vision, to assist businesses in innovating and scaling their AI projects.

### Tesla AI
Tesla AI concentrates on the development of AI-powered autonomous driving technology, pushing the boundaries of what is achievable in the automotive industry.

## Noteworthy News in Artificial Intelligence

### Breakthroughs in AI Research
Recent advances in AI research have led to breakthroughs in fields like reinforcement learning, computer vision, and neural networks, expanding the horizons of AI capabilities.

### New Partnerships and Acquisitions in the AI Landscape
Companies are forging strategic partnerships and making acquisitions to bolster their AI capabilities and broaden their market reach.

### Impact of AI on the Job Market and Society
The impact of AI on the job market and society remains a subject of ongoing discourse, touching on concerns regarding job displacement and the ethical implications of AI technologies.

### Government Regulations and Policies on AI Technology
Governments worldwide are formulating regulations and policies to govern the utilization of AI technology, ensuring responsible and ethical deployment.

## Target Audience Analysis

Professionals in the tech industry seeking insights on AI trends, business owners interested in integrating AI into their operations, students and researchers exploring AI advancements, and policymakers and regulators involved in AI governance will find this blog post informative and enlightening.

## Call to Action

Stay abreast of AI news by subscribing to newsletters or following industry experts. Engage in discussions on AI topics through social media platforms or online forums to remain informed and contribute to the dialogue.

In conclusion, Artificial Intelligence stands as a rapidly evolving field with vast potential to reshape industries and enhance our daily lives. By remaining informed about the latest trends and advancements in AI, we can harness the capabilities of this technology to foster innovation and create a better future for all.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [20]:
topic = "2024 Olympics"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on 2024 Olympics.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Unveiling the Excitement: A Sneak Peek into the 2024 Olympics

Introduction:
- Brief overview of the 2024 Olympics, including the host city, dates, and key events
- Highlight the significance of the Olympics in the world of sports and international relations

Key Points:
1. Latest Trends:
- Sustainability initiatives in the planning and execution of the 2024 Olympics
- Emphasis on gender equality and diversity in sports participation
- Integration of technology for enhanced viewer experience and

I now can give a great answer

Final Answer: 

# Unveiling the Excitement: A Sneak Peek into the 2024 Olympics

The anticipation for the 2024 Olympics is reaching a fever pitch as the world gears up for this prestigious event. Set to take place in the vibrant city of Paris, the 2024 Olympics promise to showcase the best of sportsmanship, athleticism, and international camaraderie. Scheduled to happen from July 26 to August 11, 2024, this global sporting extravaganza will bring together athletes from around the world to compete in a variety of disciplines, from track and field to swimming to gymnastics. The Olympics hold a special place in the hearts of sports enthusiasts and spectators alike, as they symbolize unity, excellence, and the pursuit of athletic achievement. Beyond the thrill of competition, the Olympics also serve as a platform for countries to come together, transcend differences, and celebrate the spirit of sportsmanship on a global stage.

In line with the growing focus 

In [21]:
Markdown(result)

# Unveiling the Excitement: A Sneak Peek into the 2024 Olympics

The anticipation for the 2024 Olympics is reaching a fever pitch as the world gears up for this prestigious event. Set to take place in the vibrant city of Paris, the 2024 Olympics promise to showcase the best of sportsmanship, athleticism, and international camaraderie. Scheduled to happen from July 26 to August 11, 2024, this global sporting extravaganza will bring together athletes from around the world to compete in a variety of disciplines, from track and field to swimming to gymnastics. The Olympics hold a special place in the hearts of sports enthusiasts and spectators alike, as they symbolize unity, excellence, and the pursuit of athletic achievement. Beyond the thrill of competition, the Olympics also serve as a platform for countries to come together, transcend differences, and celebrate the spirit of sportsmanship on a global stage.

In line with the growing focus on sustainability and environmental responsibility, the 2024 Olympics are set to incorporate various initiatives to minimize their carbon footprint and promote eco-friendly practices. From using renewable energy sources to implementing waste reduction strategies, the organizers are committed to making the event as environmentally friendly as possible. Moreover, the 2024 Olympics are championing gender equality and diversity in sports participation, aiming to provide equal opportunities for athletes of all genders and backgrounds. By promoting inclusivity and representation, the Olympics are paving the way for a more equitable and inclusive sporting landscape. The integration of technology is another key trend shaping the 2024 Olympics, with a focus on enhancing the viewer experience and improving athlete performance. From virtual reality experiences for spectators to advanced training tools for athletes, technology is playing a pivotal role in elevating the overall Olympic experience for all stakeholders.

At the helm of the 2024 Olympics is the International Olympic Committee (IOC), responsible for organizing and overseeing the event to ensure its smooth and successful execution. Working closely with the IOC are the National Olympic Committees, which play a crucial role in selecting and supporting athletes to represent their respective countries at the Olympics. Additionally, the 2024 Olympics are made possible thanks to the generous support of sponsors and partners who provide financial and logistical assistance to bring the event to life. Their contributions help fund infrastructure development, athlete training programs, and promotional activities, ensuring that the Olympics are a world-class sporting spectacle. As the countdown to the 2024 Olympics continues, updates on venue construction and preparation progress are a hot topic of discussion. From the iconic venues in Paris to the state-of-the-art facilities being built for the athletes, every detail contributes to the excitement and anticipation surrounding the event.

Athlete profiles and stories leading up to the Olympics are also capturing the attention of sports enthusiasts, offering a glimpse into the journeys and aspirations of the world's top athletes. These personal narratives add a human touch to the competitive spirit of the Olympics, showcasing the dedication, perseverance, and talent of the athletes who will compete on the global stage. While the road to the 2024 Olympics is paved with excitement and anticipation, potential controversies or challenges may arise along the way. From logistical hurdles to political tensions, navigating the complexities of organizing a multi-sport event of this scale requires foresight, adaptability, and collaboration from all stakeholders involved. In conclusion, the 2024 Olympics are set to captivate audiences worldwide with their blend of athleticism, sportsmanship, and international unity. From sustainability initiatives to gender equality efforts to technological advancements, the Olympics are embracing the latest trends and innovations to deliver a memorable and impactful sporting experience. As the world eagerly awaits the opening ceremony in Paris, the stage is set for the 2024 Olympics to inspire, unite, and celebrate the power of sports on a global scale. Stay tuned for more updates and insights on the 2024 Olympics by subscribing to our blog or following us on social media. Join the conversation and share your thoughts and predictions for the upcoming Olympics as we countdown to this historic event. Let the games begin!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).